In [69]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [73]:
url = "https://github.com/ayedaemon/RuckSack-Python/blob/master/log_analysis/access_log"
url1 = "https://github.com/ayedaemon/RuckSack-Python/blob/master/log_analysis/access_log2"
url2 = "https://github.com/robert456456456456/Web_server_log_parser/blob/master/devops.log"

In [74]:
def html_to_df(url, clean_empty = False , attrs = {}, helper = None):
    html_content = requests.get(url).text
    soup = bs(html_content, "lxml")
    table = str(soup.find("table", attrs=attrs))
    df = pd.read_html(table)[0]
    if clean_empty :
        df = df.loc[:, ~df.isnull().all(axis = 0)]
    if helper:
        df = helper(df)   
    return df

In [75]:
df1 = html_to_df(url,clean_empty=True)
df2 = html_to_df(url1,clean_empty=True)
df = pd.concat([df1, df2],ignore_index=True)

In [76]:
def log_parser(str_):
    find = {}
    find['User Address'] = str_.split()[0]
    find['RFC931'] = str_.split()[1]
    find['User'] = str_.split()[2]
    find['Date'] = str_.split('[')[1].split()[0]
    find['GMT'] = str_.split('[')[1].split()[1].strip(']')
    try:
        if 'HTTP' in str_.split('"')[1].split()[-1]:
            find['Action'] =  str_.split('"')[1].replace(str_.split('"')[1].split()[2],'').strip()
        else:
            find['Action'] =  str_.split('"')[1].strip()
    except:
        find['Action'] = '-'
    try:
        find['Status'] = str_.split('"')[2].strip().split()[0]
    except:
        find['Status'] = '-'
    try:
        find['Size'] = str_.split('"')[2].strip().split()[1]
    except:
        find['Size'] = '-'
    try:
        find['Referrer'] = str_.strip().split('"')[3]
    except:
        find['Referrer'] ='-'
    try:
        find['Browser'] = str_.strip().split('"')[5]
    except:
        find['Browser'] = '-'
        
    return find

In [77]:
def log_df(df, col_name, columns = [] ):
    df = pd.DataFrame(list(df[col_name].apply(log_parser).values) )
    if len(df.columns) == len(columns):
        df.columns = columns
    return df

In [78]:
columns = ["User Address","RFC931","User","Date","GMT","Action","Status","Size","Referrer","Browser"]
df = log_df(df, 1, columns)
df.head()

,User Address,RFC931,User,Date,GMT,Action,Status,Size,Referrer,Browser
0,127.0.0.1,-,-,15/Jan/2021:08:54:07,+0000,GET /,403,4006,-,curl/7.61.1
1,127.0.0.1,-,-,15/Jan/2021:08:55:23,+0000,GET /cgi-bin/weakform.py,403,199,-,curl/7.61.1
2,127.0.0.1,-,-,15/Jan/2021:09:01:38,+0000,GET /cgi-bin/weak_form.py,200,19,-,curl/7.61.1
3,172.17.0.1,-,-,15/Jan/2021:09:02:14,+0000,GET /,403,4288,-,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...
4,172.17.0.1,-,-,15/Jan/2021:09:02:15,+0000,GET /noindex/common/css/bootstrap.min.css,200,99548,http://172.17.0.2/,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...


In [79]:
def find_XSS(str_):
    flag = 0
    l1 = ['<','>','\\','`']
    l2 = ['/',')','(']
    for char in l1:
        ascii_encoding = '%' + hex(ord(char)).replace('0x', '')
        if (char in str_) or (ascii_encoding in str_):
            flag = 1
            break
    if flag != 1:
        try:
            req_str = ''.join(str_.split('?')[1 : ])
            for char in l2:
                ascii_encoding = '%' + hex(ord(char)).replace('0x', '')
                if (char in req_str) or (ascii_encoding in req_str):
                    flag = 1
                    break
        except:
            pass
    return bool(flag)

In [80]:
df['XSS Attack'] = df['Action'].apply(find_XSS)

In [81]:
df[df["XSS Attack"] == True].count()

User Address    48
RFC931          48
User            48
Date            48
GMT             48
Action          48
Status          48
Size            48
Referrer        48
Browser         48
XSS Attack      48
dtype: int64